In [1]:
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import numpy as np

### A/B тестирование

В этом задании вы познакомитесь с A/B тестированием и примените полученные знания по статистике. 

Рассмотрим A/B тестирование на примере сайта. У сайта есть два дизайна - старый и новый, и мы хотим оценить, насколько новый дизайн лучше старого. Для этого пользователи сайта случайным образом разделяются на контрольную и тестовую группы. Контрольной группе показывается старая версия сайта, тестовой группе - измененная версия. Оценить изменение можно несколькими способами, самый простой - оценить конверсию. Конверсия - доля пользователей, совершивших заранее определенное действие(например подписка, нажатие на кнопку, заполнение формы).

### Описание данных

Для начала нужно загрузить данные из файла `a_b_testing.csv` при помощи функции `read_csv` из библиотеки `pandas`. В данном случае 1 - была совершена подписка на сайт, 0 - подписки не было. A - контрольная группа, B - тестовая группа.

In [2]:
import numpy as np
from scipy import stats
import pandas as pd

abtest = pd.read_csv('a_b_testing.csv')

In [3]:
abtest.head()

,converted,group
0,1,A
1,0,A
2,0,A
3,1,B
4,1,A


In [4]:
counts = abtest['converted'].value_counts()
counts[1]

1697

In [5]:
df = abtest['converted'].value_counts().reset_index()
df.columns = ['col', 'count']
df

,col,count
0,0,2303
1,1,1697


Далее нужно выполнить следующие пункты, описание выходного формата содержится внутри каждого задания.

### Доверительный интервал

В видео про доверительный интервал мы рассмотрели, как можно аппроксимировать биномиальное распределение нормальным. В некоторых случаях параметры нормального распределения можно вывести математически и ниже мы рассмотрим как.

Представим количество пользователей как случайную величину из биномиального распределения с параметрами  `n`  - количество пользователей и `p` - вероятность конверсии или как сумму  `n`  независимых бросаний монетки. Определим следующую случайную величину:

$$Y = X_{1} + X_{2} + \dots + X_{n} , \, $$
где случайная величина $X_{i}$ имеет распределение Бернулли. Для случайной величины $Y$ математическое ожидание и дисперсия равны:

$$\mu = np, \, \sigma^{2} = np\cdot(1 - p)$$

Далее применяя центральную предельную теорему(случайные величины $X_{i}$ распределены независимо и размер выборки большой), получаем что 

$$Y \sim \mathcal{N}(np \, np\cdot(1 - p))\$$

Мы перешли от биномиального распределения к нормальному. Следующий шаг - стандартизация нормального распределения:

$$Z = \frac{Y - np}{\sqrt{np\cdot(1-p)}} \sim \mathcal{N}(0, \, 1) $$ 

Преобразуем выражение выше:

$$Z = \frac{Y - np}{\sqrt{np\cdot(1-p)}} = \frac{\frac{Y}{n} - p}{\sqrt{\frac{p(1-p)}{n}}} \sim \mathcal{N}(0, \, 1) $$

Так как среднее значение по выборке - это наблюдаемый процент конверсии, то доверительный интервал будет выглядеть следующим образом:

$${P}\left(p - z_{1-\frac{\alpha}{2}} \sqrt{\frac{p(1-p)}{n}} \le \mu \le p + z_{1-\frac{\alpha}{2}}\sqrt{\frac{p(1-p)}{n}}\right) = 1-\alpha$$

### ЗАДАНИЕ

Найдите доверительный интервал для средней конверсии пользователей из контрольной выборки с уровнем значимости 95%. Округлите левую и правую границу с точностью до двух знаков после запятой. Запишите значения левой и правой границ через запятую, сохраняя приведенный порядок, в переменную `answer1`, которая будет являться строкой.




Оказывается в первом вопросе нужно вписать доверительный интервал не для "среднего кол-ва пользователей", а для вероятности подписки.

#### РЕШЕНИЕ

In [56]:
A = abtest[abtest['group'] == 'A']
A

,converted,group
0,1,A
1,0,A
2,0,A
4,1,A
7,0,A
...,...,...
3993,1,A
3994,0,A
3995,1,A
3996,0,A


In [57]:
counts = A['converted'].value_counts()
counts[1] # совершили действие

839

In [58]:
sample_size = len(A)
sample_size

2037

In [59]:
p = counts[1] / sample_size
p

0.4118802160039273

In [60]:
def compute_ci(sample, sample_mean, st_dev):
    z_value = stats.norm.ppf(q = 0.95)
    sample_size = len(sample)
    interval = z_value * np.sqrt(st_dev / sample_size)
    conv_inv = (round(sample_mean - interval, 2), round(sample_mean + interval,2))
    
    return conv_inv

In [61]:
sample = A['converted']
st_dev = p * (1 - p)

ci = compute_ci(sample, p, st_dev)

print('conf interval:', ci)

conf interval: (0.39, 0.43)


In [62]:
st_dev

0.2422349036684855

### Задача A/B тестирования

Рассмотрим независимые выборки $X$ и $Y$ для которых есть $\mu_x$ и $\mu_y$, определяющие среднее значение распределения.

Рассматривается следующая гипотеза:
$$
H_0: \mu_x = \mu_y
$$
против альтернативы:

$$
H_1: \mu_x \ne \mu_y.
$$

Если гипотеза $H_0$ отвергается, то показатель действительно поменялся.

Также можно тест можно записать и другим способом:
$$
H_0: \mu_x \le \mu_y
$$

против альтернативы:

$$
H_1: \mu_x > \mu_y
$$

### Задание по статистике Стьюдента 

Найдите значение статистики Стьюдента в предположении независимости выборок по формуле:

$$
T(X, Y) = \frac{\bar{X} - \bar{Y}}{\sqrt{\frac{s_x^2}{n} + \frac{s_y^2}{m}}}
$$

где `n` - размер контрольной выборки, `m`  - размер тестовой выборки.

Ответ запишите в переменную `answer2` с точностью до двух знаков после запятой.

### РЕШЕНИЕ

In [11]:
A = abtest[abtest['group'] == 'A']
A

,converted,group
0,1,A
1,0,A
2,0,A
4,1,A
7,0,A
...,...,...
3993,1,A
3994,0,A
3995,1,A
3996,0,A


In [12]:
B = abtest[abtest['group'] == 'B']
B

,converted,group
3,1,B
5,1,B
6,0,B
8,1,B
16,0,B
...,...,...
3976,1,B
3983,1,B
3985,0,B
3997,1,B


In [13]:
muA = np.mean(A['converted'])
muB = np.mean(B['converted'])

In [14]:
muA

0.4118802160039273

In [15]:
muB

0.4370860927152318

In [16]:
n = len(A)
n

2037

In [17]:
m = len(B)
m

1963

In [18]:
sA = np.var(A['converted'])
sA

0.24223490366848552

In [19]:
sB = np.var(B['converted'])
sB

0.24604184027016357

In [20]:
znam = np.sqrt((sA / n) + (sB / m))
chisl =  muA - muB
answer3 = chisl / znam
round(answer3, 2)

-1.61

### Статистика Стьюдента из библиотеки Scipy

Найдите p-value для статистики Стьюдента, используя функцию `stats.ttest_ind`.

### РЕШЕНИЕ

In [21]:
from scipy.stats import ttest_ind

answer3 = stats.ttest_ind(A['converted'].sample(n=2037), B['converted'].sample(n=1963))
answer3

Ttest_indResult(statistic=-1.6126205013707797, pvalue=0.10690590820177126)

In [22]:
round(answer3[1], 2)

0.11

Дополнительная проверка: значение статистики Стьюдента, посчитанная двумя способами, должны совпадать.

Ответ запишите в переменную `answer3` с точностью до 2 знака после запятой